In [9]:
from datetime import datetime as dt

bbox = [29.504624, -0.166193, 29.667978, -0.043299]

max_cloud_cover = 15
start_date = dt(2022, 10, 1)
end_date = dt(2023, 1, 1)

In [104]:
%run _downloads.ipynb
%run _imagery.ipynb


sub_dir = './testingDask'
dst_dir = './testingDask'

collection = get_collection(start_date, end_date, bbox, f'{dst_dir}/s2_collection.json', max_cloud_cover=max_cloud_cover)


	./testingDask/s2_collection.json: 5 items found
	{'QV': 2, 'QA': 3}


In [105]:
%run _constants.ipynb

original_scenes_dict = download_original_imagery(collection, bbox, S2_BANDS, sub_dir, overwrite=True)


	downloading... S2B_35MQV_20221203_0_L2A
	downloading... S2B_35NQA_20221203_0_L2A
	downloading... S2B_35NQA_20221103_0_L2A
	downloading... S2A_35MQV_20221009_0_L2A
	downloading... S2A_35NQA_20221009_0_L2A


In [106]:
%%time

%run _masking.ipynb

masked_scenes_dict = {}
for scene in original_scenes_dict:
    print(f'\tmasking... {scene}')
    scene_dir = f'{dst_dir}/{scene}'
    scene_dict = original_scenes_dict[scene]
    masked_scenes_dict[scene] = save_cloud_masked_images(scene_dict, scene_dir, overwrite=True)  
    

	masking... S2B_35MQV_20221203_0_L2A
	masking... S2B_35NQA_20221203_0_L2A
	masking... S2B_35NQA_20221103_0_L2A
	masking... S2A_35MQV_20221009_0_L2A
	masking... S2A_35NQA_20221009_0_L2A
CPU times: user 84.3 ms, sys: 10.1 ms, total: 94.4 ms
Wall time: 158 ms


In [112]:
%%time

%run _imagery.ipynb

band = "B08"
masked_paths = [masked_scenes_dict[scene][band] for scene in masked_scenes_dict]
stack_path = create_band_stack(band, masked_paths, dst_dir)


new stack
['-o', './testingDask/B08_stack.tif', '-separate', '-ot', 'UInt16', './testingDask/S2B_35MQV_20221203_0_L2A/B08_masked.tif', './testingDask/S2B_35NQA_20221203_0_L2A/B08_masked.tif', './testingDask/S2B_35NQA_20221103_0_L2A/B08_masked.tif', './testingDask/S2A_35MQV_20221009_0_L2A/B08_masked.tif', './testingDask/S2A_35NQA_20221009_0_L2A/B08_masked.tif']
0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 125 ms, sys: 43.4 ms, total: 168 ms
Wall time: 3.19 s


In [114]:
%%time

%run _imagery.ipynb

band = "B08"
masked_paths = [masked_scenes_dict[scene][band] for scene in masked_scenes_dict]
stack_path = create_band_stack2(band, masked_paths, dst_dir)


CPU times: user 511 ms, sys: 341 ms, total: 852 ms
Wall time: 2.02 s


In [58]:
%%time

stack_path = f"./testingDask/{band}_stack.tif"
create_composite(band, stack_path, dst_dir, method="median", overwrite=True) 

CPU times: user 4.9 s, sys: 260 ms, total: 5.16 s
Wall time: 6.46 s


'testingDask/B08_composite.tif'

In [119]:
%%time
%run _imagery.ipynb

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

stack_path = f"./testingDask/{band}_stack.tif"

composite_path = create_composite_dask(band, stack_path, dst_dir, method="median")

print(composite_path)

./testingDask/B08_composite.tif
CPU times: user 2.76 s, sys: 371 ms, total: 3.14 s
Wall time: 2.71 s


In [53]:
stack = rioxarray.open_rasterio(stack_path, chunks=(5, 1000, 1000))
#centre = stack.median(axis=0)

#entre = np.rint(centre).astype(np.uint16)
#data = centre.compute()
#data

array([3508,    0,    0,    0,    0], dtype=uint16)